In [2]:
%matplotlib inline

import os
import sys
import math
import pandas as pd
import numpy as np
import multiprocessing as mp
import matplotlib.pyplot as plt
from functools import partial

module_path = os.path.abspath(os.path.join('..'))
src_path = '{}/src'.format(module_path)
if module_path not in sys.path:
    sys.path.append(src_path)
    
from preprocessing import select, discretize
from consts import dataset, model_cols

plt.rcParams.update({'figure.max_open_warning': 0})

# Preparando a base de dados.
## Selecionando os atributos identificados como relevantes.

- Primeiro vamos definir as colunas a serem utilizadas;
- Depois, vamos utilizar
pequenos chunks para processar a base;
- Por fim, vamos salvar o arquivo
processado;
- No caso de já existir um arquivo CSV, ele será carregado.

In [4]:
save_path = '{}/data/processed/database.csv'.format(module_path)
load_path ='{}/data/raw/pessoas.txt'.format(module_path)

try:
    df_database = pd.read_csv(save_path)
except FileNotFoundError:
    # Preparando para processar o arquivo original.
    chunksize = 500
    pool = mp.Pool(mp.cpu_count())
    reader = pd.read_fwf(
        load_path,
        widths=dataset['col_widths'],
        names=dataset['col_names'],
        chunksize=chunksize
    )

    # Processamento do arquivo original.
    partial_select = partial(select, cols=model_cols)
    df_database = pd.concat(pool.map(partial_select, reader))
    df_database = df_database.replace('^\.$', '', regex=True)

    # Salvando arquivo processado.
    df_database.to_csv(save_path, header=True, index=False)

# Dimensões da base
df_database.shape
df_database.query('Diagnosticado_Depressao == 2').shape

(55967, 68)

### Selecionando instâncias diagnosticadas com depressão

- Primeiro, será feita uma query buscando as instâncias onde o campo Diagnosticado_Depressao é igual 1;
- Depois, o resultado será salvo em um novo arquivo csv;
- Caso o arquivo já exista, basta ser carregado.

In [3]:
df_dep = df_database.query('Diagnosticado_Depressao == 1')
df_dep.shape

(4235, 68)

### Selecionando instâncias não diagnosticadas com depressão

- Primeiro, será feita uma query buscando as instâncias onde o campo Diagnosticado_Depressao é igual 2;
- Depois, o resultado será salvo em um novo arquivo csv;
- Caso o arquivo já exista, basta ser carregado.

In [4]:
df_ndep = df_database.query('Diagnosticado_Depressao == 2').sample(df_dep.shape[0])
df_ndep.shape

(4235, 68)

### Unindo as instâncias

O objetivo é trabalhar com instâncias diagnosticadas ou não com depressão. Para tanto, iremos concatenar as duas bases geradas.

In [3]:
dataset_path = '{}/data/processed/nd_dataset.csv'.format(module_path)
try:
    df_dataset = pd.read_csv(dataset_path)
except FileNotFoundError:
    df_dataset = pd.concat([df_dep, df_ndep])
    df_dataset.to_csv(dataset_path, header=True, index=False)
    df_dataset.shape

### Separando amostra de 1000 instâncias para análise

In [6]:
sample_path = '{}/data/processed/sample.csv'.format(module_path)
try:
    df_sample = pd.read_csv(sample_path)
except FileNotFoundError:
    df_sample = df_dataset.sample(1000)
    df_sample.to_csv(sample_path, header=True, index=False)
    df_sample.shape

### Pré-processamento da base

(a) Primeiro iremos remover os _outliers_

In [4]:
# Após a primeira passada, instâncias que antes não eram consideradas outliers podem passar a ser.
# Por esse motivo, precisamos rodar até que não haja alteração na base.
comparator = -1
nrows = df_dataset.shape[0]

while comparator != nrows:
    comparator = nrows
    
    for col in ['Idade', 'Parou_Menstruar', 'Uso_Quantos_Dias_Remedio_Dormir', 'Idade_Gravidez']:
        df_outliers = df_dataset[col]
        limiar = 3 * df_outliers.std()
        mean = df_outliers.mean()
        outliers = (df_outliers < mean - limiar) | (df_outliers > mean + limiar)

        # Recupera apenas instâncias positivas (outliers).
        outliers = outliers[outliers]

        # Remove instâncias que são outliers.
        df_dataset = df_dataset.drop(outliers.index)
        
    nrows = df_dataset.shape[0]
    
print(df_dataset.shape)

KeyError: 'Parou_Menstruar'

(b) Agora, removeremos o atributo "Idade de Gravidez", uma vez que seu poder discriminativo é inferior ao limiar de 0,5

In [5]:
df_dataset = df_dataset.drop(['Idade_Gravidez'], axis=1)
print(df_dataset.shape)

KeyError: "['Idade_Gravidez'] not found in axis"

(c) Removendo atributos com porcentagem de dados ausentes > 80

In [9]:
n = df_dataset.shape[0]
missing_vals = ((n - df_dataset.count()) / df_dataset.shape[0]) * 100
cols = { 0: 'Porcentagem' }

limiar = missing_vals[missing_vals > 80]
print(len(limiar.index.tolist()))

df_dataset = df_dataset.drop(limiar.index.tolist(), axis=1)
df_dataset.shape

32


(8290, 35)

(d) Alguns atributos são relacionados especificamente ao sexo feminino. Por esse motivo, o volume de dados ausentes pode ser elevado. Assim, Iremos agrupar os atributos que se assemelham (quando existentes) e associá-los com o sexo da seguinte forma:

  - 0: Sexo masculino;
  - 1: Sexo feminino + atributo positivo;
  - 2: Sexo feminino + atributo negativo;
  
&nbsp; &nbsp; &nbsp; &nbsp; (i) Aborto Espontâneo e Provocado => Aborto/Sexo

In [10]:
df_dataset.loc[df_dataset.Sexo == 1, 'Aborto'] = 0
df_dataset.loc[(df_dataset.Sexo == 2)
               & ((df_dataset.Aborto_Espontaneo == 1)
                  | (df_dataset.Aborto_Provocado == 1)), 'Aborto'] = 1
df_dataset.loc[df_dataset['Aborto'].isnull(), 'Aborto'] = 2

# Drop old attributes.
df_dataset = df_dataset.drop(['Aborto_Espontaneo', 'Aborto_Provocado'], axis=1)

df_dataset['Aborto'].count()

8290

&nbsp; &nbsp; &nbsp; &nbsp; (ii) Já utilizou algum método para evitar gravidez?

In [11]:
df_dataset.loc[df_dataset.Sexo == 1, 'Evitar_Gravidez'] = 0
df_dataset['Evitar_Gravidez'].count()

5424

&nbsp; &nbsp; &nbsp; &nbsp; (iii) Já ficou grávida?

In [12]:
df_dataset.loc[df_dataset.Sexo == 1, 'Ja_Engravidou'] = 0
df_dataset['Ja_Engravidou'].count()

6066

&nbsp; &nbsp; &nbsp; &nbsp; (iii) Ainda menstrua + parou de menstruar + entrou em menopausa ==> ainda menstrua

In [13]:
df_dataset.loc[df_dataset.Sexo == 1, 'Ainda_Menstrua'] = 0
df_dataset.loc[(df_dataset.Sexo == 2)
               & (df_dataset.Entrou_Menopausa == 1), 'Ainda_Menstrua'] = 1
df_dataset = df_dataset.drop(['Parou_Menstruar', 'Entrou_Menopausa'], axis=1)

df_dataset['Ainda_Menstrua'].value_counts()

1.0    4748
0.0    2763
2.0     183
Name: Ainda_Menstrua, dtype: int64

(e) Iremos unificar, também, outros atributos gerais com quantidade alta de dados ausentes:  
&nbsp; &nbsp; &nbsp; &nbsp; (i) Uso de remédio para dormir + Quantidade de dias;

In [14]:
# Se o indivíduo não usou remédio para dormir as últimas duas semanas (valor 2), então iremos setar a quantidade
# de dias como 0.
df_dataset.loc[df_dataset.Uso_Remedio_Dormir == 2, 'Uso_Quantos_Dias_Remedio_Dormir'] = 0
df_dataset = df_dataset.drop('Uso_Remedio_Dormir', axis=1)
df_dataset.shape

(8290, 31)

&nbsp; &nbsp; &nbsp; &nbsp; (i) Ativ_Fisica_3meses + Ativ_Fisica_Semana => Ativ_Fisica (desconsideraremos qual atividade física)

In [15]:
# Se não praticou atividade física nos últimos meses, #atividade_fisicas_na_semana = 0
df_dataset.loc[df_dataset.Ativ_Fisica_3meses == 2, 'Ativ_Fisica_Semana'] = 0

df_dataset = df_dataset.drop(['Ativ_Fisica_3meses', 'Ativ_Fisica_Qual'], axis=1)

df_dataset['Ativ_Fisica_Semana'].count()

8290

### Iremos remover os atributos "Toma_Remedio_Hipertensao" e "Execício Pesado", pelo alto volume de dados ausentes. Além disso, vamos remover o atributo 'Evitar_Gravidez', pelo alto volume de dados ausentes e por já existir na base muitos atributos relacionados exclusivamente ao sexo feminino.

In [16]:
df_dataset = df_dataset.drop(['Toma_Remedio_Hipertensao', 'Exercicio_Pesado', 'Evitar_Gravidez'], axis=1)
df_dataset.shape

(8290, 26)

### Vamos, ainda, remover as instâncias com dados ausentes restantes

In [17]:
for col in list(df_dataset):
    df_dataset = df_dataset.drop(df_dataset[df_dataset[col].isnull()].index.tolist())
   
df_dataset.shape

(5236, 26)

### Por fim, vamos remover os atributos "Região", uma vez que é usado apenas para análises

In [18]:
df_dataset = df_dataset.drop(['Regiao'], axis=1)
df_dataset.shape

(5236, 25)

In [4]:
n = df_dataset.shape[0]
missing_vals = ((n - df_dataset.count()) / df_dataset.shape[0]) * 100
cols = { 0: 'Porcentagem' }

select(missing_vals.to_frame(), cols)

,Porcentagem
Diagnosticado_Depressao,0.0
Defic_Intelectual,0.0
Doença_Fisica_Cronica_Mental,0.0
Esqz_Bipol_Psicose_TOC,0.0
Diagnostico_Doenca_Coracao,0.0
Diagnostico_Doença_Pulmao,0.0
Diagnostico_Cancer,0.0
Diagnostico_Insuf_Renal_Cronica,0.0
Duas_Semanas_Cansado,0.0
Duas_Semanas _Problema_Sono,0.0


## Discretizando atributos numéricos

In [7]:
df_final = df_dataset.copy()

# Idade.
bins = np.linspace(df_dataset['Idade'].min(), df_dataset['Idade'].max() + 1, 6, dtype=int)
df_final['Idade'] = discretize(df_dataset, 'Idade', bins, labels=[i for i in range(len(bins) - 1)])

# Qtd. dias de uso remédio para dormir
bins = np.linspace(df_dataset['Uso_Quantos_Dias_Remedio_Dormir'].min(), 
                   df_dataset['Uso_Quantos_Dias_Remedio_Dormir'].max() + 1, 4, dtype=int)

df_final['Uso_Quantos_Dias_Remedio_Dormir'] = discretize(df_dataset, 'Uso_Quantos_Dias_Remedio_Dormir', 
                                                         bins,
                                                        labels=[i for i in range(len(bins) - 1)])
df_final[['Idade', 'Uso_Quantos_Dias_Remedio_Dormir']].head(5)

,Idade,Uso_Quantos_Dias_Remedio_Dormir
0,0,1
1,4,1
2,2,0
3,1,2
4,0,0


## Salvando a base final

In [8]:
# Sem discretização
non_discretized = dataset_path = '{}/data/processed/nd_dataset.csv'.format(module_path)
df_dataset.to_csv(non_discretized, header=True, index=False)

# Com discretização + diag. depressão
discretized_dp = dataset_path = '{}/data/processed/d_dataset_dp.csv'.format(module_path)
df_final.to_csv(discretized_dp, header=True, index=False)

# Com discretização - diag. depressão
discretized_ndp = dataset_path = '{}/data/processed/d_dataset_ndp.csv'.format(module_path)
df_final.drop('Diagnosticado_Depressao', axis=1).to_csv(discretized_ndp, header=True, index=False)


df_final.shape

(5236, 25)